In [1]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
init_printing(use_unicode=True, wrap_line=False)

In [3]:
x, y, z = symbols("x y z")

In [4]:
def GradDescend(f, x0:np.ndarray, lr:float = 0.1, eps:float = 1e-06) -> np.ndarray:
    #Считаем производные прямо здесь
    _dx = [diff(f,x), diff(f,y)]
    res = []
    while True:
        dx = np.array([float(_dx[0].subs(x, x0[0])), float(_dx[1].subs(y, x0[1]))])
        x0 = x0 - lr * dx
        res.append(x0)
        if np.linalg.norm(dx) < eps:
            break
    return res

In [5]:
def GradDesDynamicLR(f, x0:np.ndarray, lr:float = 0.1, eps:float = 1e-06, T:float = 80) -> np.ndarray:
    _dx = [diff(f,x), diff(f,y)]
    res = []
    t = 0
    while True:
        dx = np.array([float(_dx[0].subs(x, x0[0])), float(_dx[1].subs(y, x0[1]))])
        x0 = x0 - lr * dx
        res.append(x0)
        #dynamic learning rate
        t += 1./T
        lr *= (1 - t / T)
        if np.linalg.norm(dx) < eps:
            break
        elif t >= T:
            break
    return res

In [6]:
def GradDesImpulse(f, x0:np.ndarray, lr:float = 0.1, eps:float = 1e-06) -> np.ndarray:

    return 0

In [7]:
def Adadelta(f, x0:np.ndarray, lr:float = 0.1, eps:float = 1e-06) -> np.ndarray:
    
    return 0

In [8]:
function = x**2 + y**2

In [ ]:
out1 = np.array(GradDesDynamicLR(function, np.array([1, 1])))

ValueError: 
Can't calculate derivative wrt -1.00000000000000.

In [ ]:
def f(x, y):
    return x ** 2 + y ** 2

In [ ]:
bounds = np.asarray([[-1.0, 1.0], [-1.0, 1.0]])
xaxis = np.arange(bounds[0,0], bounds[0,1], 0.1)
yaxis = np.arange(bounds[1,0], bounds[1,1], 0.1)
x, y = np.meshgrid(xaxis, yaxis)

results = f(x, y)
plt.contourf(x, y, results, levels=60, cmap='jet')

# plot the sample as black circles
solutions = np.asarray(out1)
plt.plot(solutions[:, 0], solutions[:, 1], '.-', color='w')
plt.show()